# Assignment 2 - Portfolio Optimization

### <i> Group Members: Sayantan Raut, Dhruv Jain, Shubhankar Rekhade </i> 

In [1]:
##### Reading Data

import pandas as pd
import numpy as np
from gurobipy import *

df = pd.read_csv('Prices.csv')
df.head()

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,PWR,QCOM,DGX,RL,RJF,RTN,O,REG,SCHP,BND
0,138.0,38.2,52.0,26.9,81.1,18.1,59.3,3.95,110.0,14.3,...,31.3,73.3,52.8,175.0,51.5,88.8,37.3,46.0,52.7,80.1
1,128.0,36.7,49.2,27.5,79.9,17.1,59.2,3.43,115.0,14.1,...,31.2,74.2,52.5,157.0,50.9,95.1,40.8,48.1,54.0,81.3
2,132.0,39.3,50.1,27.7,82.5,19.5,67.9,3.67,126.0,13.9,...,35.4,73.6,52.4,160.0,52.1,98.2,44.0,50.6,54.5,81.5
3,136.0,38.5,51.4,26.0,79.7,20.4,65.7,4.01,127.0,14.3,...,36.9,78.9,57.9,161.0,55.9,98.8,40.9,51.1,54.0,81.2
4,138.0,38.5,51.4,24.0,79.7,19.7,60.9,4.03,120.0,14.3,...,34.7,78.5,55.7,152.0,49.3,95.9,43.4,52.5,54.5,81.5


In [2]:
##### Pre processing

import import_ipynb
import Assignment_2_Preprocessing as preprocessing

importing Jupyter notebook from Assignment 2 Preprocessing.ipynb


In [3]:
msft_index = df.columns.get_loc('MSFT')
gs_index = df.columns.get_loc('GS')
pg_index = df.columns.get_loc('PG')
schp_index = df.columns.get_loc('SCHP')

C = preprocessing.C

r = preprocessing.avg_return

In [4]:
# Model 1
model1 = Model()
rho = 0.005

index_arr = [msft_index, gs_index, pg_index, schp_index]

C_1 = [[C[i][j] for i in index_arr] for j in index_arr]

r_1 = [r[i] for i in index_arr]
    
w = model1.addVars(4)

# Constraints:
model1.addConstr(sum(w[i]*r_1[i] for i in range(4)) >= rho)
model1.addConstr(sum(w[i] for i in range(4)) == 1)
model1.addConstrs(w[i] >= 0 for i in range(4))

# Objective function:
model1.setObjective(sum(w[i]*w[j]*C_1[i][j] for i in range(4) for j in range(4)), GRB.MINIMIZE)

model1.update()
model1.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x24f4abf9
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal  

In [5]:
# Model 2
model2 = Model()
rho = 0.005
    
w = model2.addVars(390)

# Constraints:
model2.addConstr(sum(w[i]*r[i] for i in range(390)) >= rho)
model2.addConstr(sum(w[i] for i in range(390)) == 1)
model2.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model2.setObjective(sum(w[i]*w[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

model2.update()
model2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x509900e3
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633

In [6]:
val_arr = [(i,w[i].x) for i in range(390) if w[i].x >= 0.00001] # threshold of stocks...change RHS to improve/reduce sum value (~1)

stocks_inds = [val_arr[i][0] for i in range (len(val_arr))]
stocks_inds # stock indices

returns = [0 + val_arr[i][1] for i in range (len(val_arr))]
sum(returns)

0.9999152765661383

In [7]:
# Model 3
model3 = Model()
rho = 0.005
    
w = model3.addVars(390)
b = model3.addVars(390, vtype = GRB.BINARY)
z = model3.addVars(390)

# Constraints:
model3.addConstrs(z[i] == w[i]*b[i] for i in range(390))
model3.addConstr(sum(b[i] for i in range(390)) <= 4)
model3.addConstr(sum(z[i]*r[i] for i in range(390)) >= rho)
model3.addConstr(sum(w[i]*b[i] for i in range(390)) == 1)
model3.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model3.setObjective(sum(z[i]*z[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

model3.update()
model3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 1170 columns and 1170 nonzeros
Model fingerprint: 0x18276b8c
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 780 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.04s
Presolved: 1173 rows, 2340 columns, 3900 nonzeros
Presolved model has 780 SOS constraint(s)
Presolved model has 76245 quadratic objective terms
Variable types: 1560 continuous, 780 integer (780 binary)
Found heuristic solution:

## Question 1

### Formulate and solve each of the three models in Python, and then answer the following questions. For each part, also include your Gurobi output.

### Model 1

In [8]:
# Model 1
model1 = Model()
rho = 0.005

index_arr = [msft_index, gs_index, pg_index, schp_index]

C_1 = [[C[i][j] for i in index_arr] for j in index_arr]

r_1 = [r[i] for i in index_arr]
    
w = model1.addVars(4)

# Constraints:
model1.addConstr(sum(w[i]*r_1[i] for i in range(4)) >= rho)
model1.addConstr(sum(w[i] for i in range(4)) == 1)
model1.addConstrs(w[i] >= 0 for i in range(4))

# Objective function:
model1.setObjective(sum(w[i]*w[j]*C_1[i][j] for i in range(4) for j in range(4)), GRB.MINIMIZE)

model1.update()
model1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x24f4abf9
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.93770406e+03 -2.93770406e+0

### a) For Model 1, write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.

In [9]:
optimal_risk = model1.objVal

solver_time = model1.Runtime

optimal_weights = [(df.columns[index_arr[i]],w[i].x) for i in range(4)]

print("Optimal Risk Acieved: ", np.round(optimal_risk, 5))
print("Solver Time Taken:", np.round(solver_time,5), "s\n")

for stock, weight in optimal_weights:
    print(stock," ", np.round(weight*100,5),"%")

Optimal Risk Acieved:  0.00018
Solver Time Taken: 0.02363 s

MSFT   23.71176 %
GS   2.58596 %
PG   0.0 %
SCHP   73.70228 %


### Model 2

In [10]:
# Model 2
model2 = Model()
rho = 0.005
    
w = model2.addVars(390)

# Constraints:
model2.addConstr(sum(w[i]*r[i] for i in range(390)) >= rho)
model2.addConstr(sum(w[i] for i in range(390)) == 1)
model2.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model2.setObjective(sum(w[i]*w[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

model2.update()
model2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x509900e3
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633

In [11]:
val_arr = [(i,w[i].x) for i in range(390) if w[i].x >= 0.00001] # threshold of stocks...change RHS to improve/reduce sum value (~1)

stocks_inds = [val_arr[i][0] for i in range (len(val_arr))]
stocks_inds # stock indices

returns = [0 + val_arr[i][1] for i in range (len(val_arr))]
sum(returns)

0.9999152765661383

### b) For Model 2, write down the optimal risk and solver time.

In [12]:
optimal_risk = model2.objVal

solver_time = model2.Runtime

print("Optimal Risk Acieved: ", np.round(optimal_risk, 5))
print("Solver Time Taken:", np.round(solver_time,5), "s\n")

val_arr = [(i,w[i].x) for i in range(390) if w[i].x > 0.00001]

for k,v in val_arr:
    print(df.columns[k], " ", np.round(v*100,5),"%")

Optimal Risk Acieved:  3e-05
Solver Time Taken: 0.07611 s

ABBV   1.1364 %
ABMD   0.66541 %
ATVI   1.36743 %
ANET   1.17532 %
AIZ   2.22128 %
ATO   2.90637 %
BBY   0.26065 %
CME   1.27911 %
ED   0.04017 %
DRI   0.66555 %
RE   0.23315 %
GWW   0.77415 %
HAS   0.24506 %
HCA   2.95684 %
HUM   2.26682 %
INFO   5.17086 %
ICE   1.417 %
KEYS   2.21688 %
LHX   0.00746 %
LLY   0.8515 %
LMT   0.06075 %
PSX   0.79815 %
PNC   3.2313 %
PWR   0.00104 %
BND   68.04286 %


### Model 3

In [13]:
# Model 3
model3 = Model()
rho = 0.005
    
w = model3.addVars(390)
b = model3.addVars(390, vtype = GRB.BINARY)
z = model3.addVars(390)

# Constraints:
model3.addConstrs(z[i] == w[i]*b[i] for i in range(390))
model3.addConstr(sum(b[i] for i in range(390)) <= 4)
model3.addConstr(sum(z[i]*r[i] for i in range(390)) >= rho)
model3.addConstr(sum(w[i]*b[i] for i in range(390)) == 1)
model3.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model3.setObjective(sum(z[i]*z[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

model3.update()
model3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 1170 columns and 1170 nonzeros
Model fingerprint: 0x18276b8c
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 780 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1173 rows, 2340 columns, 3900 nonzeros
Presolved model has 780 SOS constraint(s)
Presolved model has 76245 quadratic objective terms
Variable types: 1560 continuous, 780 integer (780 binary)
Found heuristic solution:

### c) For Model 3, report the optimal risk, solver time, and the ticker and weight on each of the four stocks selected by the model.

In [14]:
val_arr = [(i,w[i].x) for i in range(390) if w[i].x > 0]
returns = [0 + val_arr[i][1] for i in range (len(val_arr))]
sum(returns)

optimal_risk = model3.objVal

solver_time = model3.Runtime

print("Optimal Risk Acieved: ", np.round(optimal_risk, 5))
print("Solver Time Taken:", np.round(solver_time,5), "s\n")

# printing as table
for k,v in val_arr:
    print(df.columns[k], " ", np.round(v*100,2),"%")

Optimal Risk Acieved:  7e-05
Solver Time Taken: 36.45018 s

CME   12.64 %
LLY   7.55 %
NVDA   4.38 %
BND   75.44 %


## Question 2

### Use your solution to Question 1 above to answer the following questions:

### a) Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.

### Ans

The optimal risk is **lower** in Model 2 as we are including all possible risk portfolio from combination across all assets. This will help optimizer choose a better portfolio from a wider range of options instead of just 4 random assets.

### b) Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences.

### Ans

The optimal risk is **slightly higher** in Model 3 compared to Model 2 because from all the possible combinations of assets, we are including a combination of just 4 assets into our risk portfolio. This will help optimizer choose from a large pool of combinations, but still won't be as exhaustive as in Model 2, due to the number restriction. Hence the optimal value observed is slightly higher.

## Question 3

### In some cases, we may want to get an approximate solution quickly by terminating the branch- and-bound algorithm before it finds an optimal solution. There are two ways to terminate Gurobi early: (a) by setting a maximum time limit, and (b) by setting a maximum acceptable optimality gap (the tolerance). Use Model 3 to answer the following two questions. For each part, also include your Gurobi output.

### a) Set Gurobi to terminate after 3 seconds by including XYZ.Params.TimeLimit = 3.0 in your code for Model 3, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?

In [15]:
# Model 3
model3 = Model()
rho = 0.005
    
w = model3.addVars(390)
b = model3.addVars(390, vtype = GRB.BINARY)
z = model3.addVars(390)

# Constraints:
model3.addConstrs(z[i] == w[i]*b[i] for i in range(390))
model3.addConstr(sum(b[i] for i in range(390)) <= 4)
model3.addConstr(sum(z[i]*r[i] for i in range(390)) >= rho)
model3.addConstr(sum(w[i]*b[i] for i in range(390)) == 1)
model3.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model3.setObjective(sum(z[i]*z[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

# Setting time limit
model3.Params.TimeLimit = 3.0

model3.update()
model3.optimize()

Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 1170 columns and 1170 nonzeros
Model fingerprint: 0x18276b8c
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 780 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1173 rows, 2340 columns, 3900 nonzeros
Presolved model has 780 SOS constraint(s)
Presolved model has 76245 quadratic objective terms
Variable types: 1560 continuous, 780 integer (78

In [16]:
optimal_risk = model3.objVal

solver_time = model3.Runtime

print("Optimal Risk Acieved: ", np.round(optimal_risk, 5))
print("Solver Time Taken:", np.round(solver_time,5), "s\n")

Optimal Risk Acieved:  7e-05
Solver Time Taken: 3.00733 s



### Ans

From the above results, we can see that the optimal value of objective function **stays the same**.

### b) Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1 in your code for Model 3, where ’XYZ’ is the name of your model. (Note: The default gap in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in question 1c)?

In [17]:
# Model 3
model3 = Model()
rho = 0.005
    
w = model3.addVars(390)
b = model3.addVars(390, vtype = GRB.BINARY)
z = model3.addVars(390)

# Constraints:
model3.addConstrs(z[i] == w[i]*b[i] for i in range(390))
model3.addConstr(sum(b[i] for i in range(390)) <= 4)
model3.addConstr(sum(z[i]*r[i] for i in range(390)) >= rho)
model3.addConstr(sum(w[i]*b[i] for i in range(390)) == 1)
model3.addConstrs(w[i] >= 0 for i in range(390))

# Objective function:
model3.setObjective(sum(z[i]*z[j]*C[i][j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

# Setting time limit
model3.Params.MIPGap = 0.1

model3.update()
model3.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 1170 columns and 1170 nonzeros
Model fingerprint: 0x18276b8c
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 780 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1173 rows, 2340 columns, 3900 nonzeros
Presolved model has 780 SOS constraint(s)
Presolved model has 76245 quadratic objective terms
Variable types: 1560 continuous, 780 integer (780

In [18]:
optimal_risk = model3.objVal

solver_time = model3.Runtime

print("Optimal Risk Acieved: ", np.round(optimal_risk, 5))
print("Solver Time Taken:", np.round(solver_time,5), "s\n")

Optimal Risk Acieved:  7e-05
Solver Time Taken: 32.78684 s



### Ans

From the above results, we can see that the optimal value of time taken by the model **goes up**.